In [ ]:
# Import the SVM model
from sklearn import svm
from sklearn.metrics import classification_report

# Import the required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from finta import TA
from pandas.tseries.offsets import DateOffset
import os
import requests
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout

%matplotlib inline

In [ ]:
# Import indicator dataframe
# Start with 1-minute... alghough we would like to attatch them all together soon.
df = pd.read_csv("../algotrader2/resources/aapl_1min_pivot_point_indicator_df.csv")

# df = pd.read_csv("../algotrader/resources/aapl_3min_pivot_point_indicator_df.csv")
# df.head()

# df = pd.read_csv("../algotrader/resources/aapl_1min_pivot_point_indicator_df.csv")
# df.head()

# Create our timestamp column as a datetime index, then save it as our index
df['timestamp'] = pd.to_datetime(df['timestamp'])

df.set_index('timestamp', inplace=True)

df.head()


In [ ]:
# X is everything except the signal column
X = df.drop('signal', axis=1)
# We should use the .shift() function so that our algorithm predicts the minute before realtime
# Drop the row with NaN values 
X = X.shift().dropna()
display(X.head())

# y is the signal column
y = df[("signal")]
display(y)

# Set start of training period
training_begin = X.index.min()
print(training_begin)

# Select ending period for the training data. Since we pulled a year's worth of data
# we will train on 9 months and then test with the rest
training_end = X.index.min() + DateOffset(months=9)
print(training_end)

# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:]
y_test = y.loc[training_end:]

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
display(X_train_scaled)
display(X_test_scaled)

### SVM

In [ ]:
# Create the classifier model
svm_model = svm.SVC()

# Fit the model to the data using X_train_scaled and y_train
# Fit is the 'training'
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
# This is predicting on the TRAINING portion of the scaled data. We will test it later with the TEST portion of the scaled data.
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

In [ ]:
# Evaluate the model's performance on the training data using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display report
print(training_report)

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
testing_signal_predictions = svm_model.predict(X_test_scaled)

In [ ]:
# Evaluate the model's ability to predict the trading signal for the testing data
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

#### Visually Compare the Actual and Predicted Returns

In [ ]:
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

predictions_df["predicted_signal"] = testing_signal_predictions

predictions_df["actual_returns"] = trading_df["actual_returns"]

predictions_df["trading_algorithm_returns"] = (
    predictions_df["actual_returns"] * predictions_df["predicted_signal"]
)

# Review the DataFrame
predictions_df.head()

In [ ]:
# Calculate and plot the cumulative returns for the `actual_returns` and the `trading_algorithm_returns`
(1 + predictions_df[["actual_returns", "trading_algorithm_returns"]]).cumprod().plot()

In [ ]:
# Evaluate the model using a classification report
training_report = classification_report(y_train, training_signal_predictions)

# Display report
print(training_report)